In [1]:
import cv2
import numpy as np
import imutils
from imutils.perspective import four_point_transform
import os
import time
from datetime import datetime
import mss
import sys
import math

In [2]:
TEMP_SCREENSHOTS_DIRECTORY = "temp_screenshots"

def take_screenshot(monitor_number):
    with mss.mss() as sct:
        file_name_and_type = "screenshot" + "_" + datetime.now().strftime("%Y_%m_%d-%H_%M_%S.%f") + ".png"

        if not os.path.isdir(TEMP_SCREENSHOTS_DIRECTORY):
            os.mkdir(TEMP_SCREENSHOTS_DIRECTORY)

        complete_file_name = os.path.join(TEMP_SCREENSHOTS_DIRECTORY, file_name_and_type)

        filename = sct.shot(mon=monitor_number, output=complete_file_name)

        return filename

def delete_screenshot(file_path):
    # Delete screenshot file.
    try:
        os.remove(file_path)
    except OSError as e:
        print ("Error deleting file: %s - %s." % (file_path, e.strerror))

    # Check if other screenshot files exists in the folder if ever. 
    # Then, delete them.
    for filename in os.listdir(TEMP_SCREENSHOTS_DIRECTORY):
        file_path = os.path.join(TEMP_SCREENSHOTS_DIRECTORY, 
            filename)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
        except OSError as e:
            print ("Error deleting file: %s - %s." % (file_path, 
            e.strerror))

def take_read_delete_screenshot(monitor_number):
    screenshot_filepath = take_screenshot(monitor_number)
    screenshot_image = cv2.imread(screenshot_filepath)
    delete_screenshot(screenshot_filepath)
    return screenshot_image

screenshot_image = take_read_delete_screenshot(monitor_number=1)
# cv2.imshow("screenshot_image", screenshot_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


with mss.mss() as sct:
    # Get information of monitor 2
    monitor_number = 1
    mon = sct.monitors[monitor_number]
    global MONITOR_WIDTH
    global MONITOR_HEIGHT
    MONITOR_WIDTH = mon["width"]
    MONITOR_HEIGHT = mon["height"]

In [4]:
COLOR_BLUE_GREEN = (232, 150, 19)
# COLOR_RED = (78, 15, 186)
COLOR_RED = (86, 13, 212)

DRAW_INITIAL_DELAY = 1000

def square(x):
    return x * x

def get_next_point(pt1, pt2, pixel_distance):
    x2 = pt2[0] 
    x1 = pt1[0]
    y2 = pt2[1]
    y1 = pt1[1]
    hypotenuse = math.sqrt(square(x2 - x1) + square(y2 - y1))
    x3 = x1 + (pixel_distance / hypotenuse) * (x2 - x1)
    y3 = y1 + (pixel_distance / hypotenuse) * (y2 - y1)
    return (round(x3), round(y3))

def draw_line(image, color, thickness, hasWait, pt1, pt2):
    cv2.line(img=image, pt1=pt1, pt2=pt2, color=color, thickness=thickness)
    width = image.shape[1]
    height = image.shape[0]
    x = int((MONITOR_WIDTH / 2) - (width / 2))
    y = int((MONITOR_HEIGHT / 2) - (height / 2))
    cv2.namedWindow("image")
    cv2.moveWindow("image", x, y)
    cv2.imshow("image", image)
    if hasWait:
        cv2.waitKey(1)

def animate_draw_line(image, color, thickness, pixel_per_millisecond, pt1, pt2):
    x2 = pt2[0] 
    x1 = pt1[0]
    y2 = pt2[1]
    y1 = pt1[1]
    hypotenuse = math.sqrt(square(x2 - x1) + square(y2 - y1))
    target_pt = pixel_per_millisecond
    while target_pt < round(hypotenuse):
        pt3 = get_next_point(pt1, pt2, target_pt)
        draw_line(image, color, thickness, True, pt1, pt3)
        target_pt = target_pt + pixel_per_millisecond
    # Make sure it reaches the end of the point.
    draw_line(image, color, thickness, True, pt1, pt2)

def animate_draw_line_multiple(image, color, thickness, pixel_per_millisecond, has_initial_delay, *pts_tuple):
    # Show image, then put initial delay.
    width = image.shape[1]
    height = image.shape[0]
    x = int((MONITOR_WIDTH / 2) - (width / 2))
    y = int((MONITOR_HEIGHT / 2) - (height / 2))
    cv2.namedWindow("image")
    cv2.moveWindow("image", x, y)
    cv2.imshow("image", image)
    if has_initial_delay:
        cv2.waitKey(DRAW_INITIAL_DELAY)

    hypotenuse_list = []
    target_pt_list = []
    isFinished_list = []

    isAllFinished = False

    for pt in pts_tuple:
        pt1 = pt[0]
        pt2 = pt[1]
        x2 = pt2[0]
        x1 = pt1[0]
        y2 = pt2[1]
        y1 = pt1[1]

        hypotenuse_list.append(square(x2 - x1) + square(y2 - y1))
        target_pt_list.append(pixel_per_millisecond)
        isFinished_list.append(False)

    while isAllFinished == False:
        for index, t in enumerate(target_pt_list):
            if t >= round(hypotenuse_list[index]):
                isFinished_list[index] = True
                # Make sure it reaches the end of the point.
                pt = pts_tuple[index]
                pt1 = pt[0]
                pt2 = pt[1]
                draw_line(image, color, thickness, True, pt1, pt2)
                
        isAllFinished = True
        for i in isFinished_list:
            if i == False:
                isAllFinished = False
                break
            
        for index, pt in enumerate(pts_tuple):
            if not isFinished_list[index]:
                pt1 = pt[0]
                pt2 = pt[1]
                pt3 = get_next_point(pt1, pt2, target_pt_list[index])
                draw_line(image, color, thickness, False, pt1, pt3)
                target_pt_list[index] = target_pt_list[index] + pixel_per_millisecond
        cv2.waitKey(1)

def animate_draw_contour(image, color, thickness, pixel_per_millisecond, has_initial_delay, contours):
    # Show image, then put initial delay.
    width = image.shape[1]
    height = image.shape[0]
    x = int((MONITOR_WIDTH / 2) - (width / 2))
    y = int((MONITOR_HEIGHT / 2) - (height / 2))
    cv2.namedWindow("image")
    cv2.moveWindow("image", x, y)
    cv2.imshow("image", image)
    if has_initial_delay:
        cv2.waitKey(DRAW_INITIAL_DELAY)

    print(contours)
    for index, c in enumerate(contours):
        if index == len(contours) - 1:
            break
        pt1 = contours[index][0]
        pt2 = contours[index + 1][0]
        animate_draw_line(image, color, thickness, pixel_per_millisecond, pt1, pt2)
    pt1 = contours[len(contours) - 1][0]
    pt2 = contours[0][0]
    animate_draw_line(image, color, thickness, pixel_per_millisecond, pt1, pt2)

def animate_draw_connect_points(image, color, thickness, pixel_per_millisecond, has_initial_delay, *points):
    # Show image, then put initial delay.
    width = image.shape[1]
    height = image.shape[0]
    x = int((MONITOR_WIDTH / 2) - (width / 2))
    y = int((MONITOR_HEIGHT / 2) - (height / 2))
    cv2.namedWindow("image")
    cv2.moveWindow("image", x, y)
    cv2.imshow("image", image)
    if has_initial_delay:
        cv2.waitKey(DRAW_INITIAL_DELAY)

    for index, pt in enumerate(points):
        if index == len(points) - 1:
            break
        pt1 = points[index]
        pt2 = points[index + 1]
        animate_draw_line(image, color, thickness, pixel_per_millisecond, pt1, pt2)
    pt1 = points[len(points) - 1]
    pt2 = points[0]
    animate_draw_line(image, color, thickness, pixel_per_millisecond, pt1, pt2)

def animate_draw_point(image, color, thickness, delay_in_millisecond, pt):
    for i in range(thickness):
        cv2.rectangle(img=image, pt1=pt, pt2=pt, color=color, thickness=i)
        width = image.shape[1]
        height = image.shape[0]
        x = int((MONITOR_WIDTH / 2) - (width / 2))
        y = int((MONITOR_HEIGHT / 2) - (height / 2))
        cv2.namedWindow("image")
        cv2.moveWindow("image", x, y)
        cv2.imshow("image", image)
        cv2.waitKey(delay_in_millisecond)

In [5]:
def crop_chest_gui(image, debug=False):
    if debug:
        width = image.shape[1]
        height = image.shape[0]
        x = int((MONITOR_WIDTH / 2) - (width / 2))
        y = int((MONITOR_HEIGHT / 2) - (height / 2))
        cv2.namedWindow("image")
        cv2.moveWindow("image", x, y)
        cv2.imshow("image", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 75, 255, cv2.THRESH_BINARY)[1]
    edged = cv2.Canny(thresh, 100, 200)
    if debug:
        width = image.shape[1]
        height = image.shape[0]
        x = int((MONITOR_WIDTH / 2) - (width / 2))
        y = int((MONITOR_HEIGHT / 2) - (height / 2))
        cv2.namedWindow("gray")
        cv2.moveWindow("gray", x, y)
        cv2.imshow("gray", gray)
        cv2.waitKey(0)
        cv2.namedWindow("thresh")
        cv2.moveWindow("thresh", x, y)
        cv2.imshow("thresh", thresh)
        cv2.waitKey(0)
        cv2.namedWindow("edged")
        cv2.moveWindow("edged", x, y)
        cv2.imshow("edged", edged)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    chest_gui_contour = None
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if len(approx) == 4:
            chest_gui_contour = approx
            break

    if chest_gui_contour is None:
        raise Exception("No Chest GUI detected.")

    chest_gui_contour_reshaped = chest_gui_contour.reshape(4, 2)
    min_x = sys.maxsize
    min_y = sys.maxsize
    max_x = -sys.maxsize
    max_y = -sys.maxsize
    for i in chest_gui_contour_reshaped:
        x = i[0]
        y = i[1]
        if x < min_x:
            min_x = x
        if x > max_x:
            max_x = x
        if y < min_y:
            min_y = y
        if y > max_y:
            max_y = y
    
    if debug:
        output = image.copy()

        corners_of_chest_gui_list = [
            (min_x, min_y),
            (min_x, max_y),
            (max_x, max_y),
            (max_x, min_y),
        ]
        pt_1 = (corners_of_chest_gui_list[0], corners_of_chest_gui_list[1])
        pt_2 = (corners_of_chest_gui_list[1], corners_of_chest_gui_list[2])
        pt_3 = (corners_of_chest_gui_list[2], corners_of_chest_gui_list[3])
        pt_4 = (corners_of_chest_gui_list[3], corners_of_chest_gui_list[0])
        animate_draw_line_multiple(output, COLOR_RED, 3, 6, True, pt_1, pt_2, pt_3, pt_4)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    chest_gui_img = image[min_y:max_y, min_x:max_x]
    if debug:
        width = chest_gui_img.shape[1]
        height = chest_gui_img.shape[0]
        x = int((MONITOR_WIDTH / 2) - (width / 2))
        y = int((MONITOR_HEIGHT / 2) - (height / 2))
        cv2.namedWindow("chest_gui_img")
        cv2.moveWindow("chest_gui_img", x, y)
        cv2.imshow("chest_gui_img", chest_gui_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    return chest_gui_img, corners_of_chest_gui_list

screenshot_image = cv2.imread("images/small_chest.png")
chest_gui_img, corners_of_chest_gui_list = crop_chest_gui(image=screenshot_image, debug=True)


: 

: 

In [6]:
def crop_chest_gui_chest_part(chest_gui_img, debug=False):
    chest_gui_img_height = chest_gui_img.shape[0]
    chest_gui_img_width = chest_gui_img.shape[1]

    # Check if it is large chest or small chest
    is_chest_large = False
    if chest_gui_img_height > chest_gui_img_width:
        is_chest_large = True

    if is_chest_large:
        chest_part_y_top = round(chest_gui_img_height / 13.47692)
        chest_part_height = round(chest_gui_img_height / 2.03248)
    else:
        chest_part_y_top = round(chest_gui_img_height / 10.15)
        chest_part_height = round(chest_gui_img_height / 3.05)

    chest_part_x_start = round(chest_gui_img_width / 27.8)
    chest_part_width = round(chest_gui_img_width / 1.075)
        
    chest_part_y_bottom = chest_part_y_top + chest_part_height
    chest_part_x_end = chest_part_x_start + chest_part_width

    if debug:
        output = chest_gui_img.copy()

        chest_part_contour = np.array([
            [chest_part_x_start, chest_part_y_top], 
            [chest_part_x_start, chest_part_y_bottom], 
            [chest_part_x_end, chest_part_y_bottom], 
            [chest_part_x_end, chest_part_y_top]
        ])

        animate_draw_connect_points(output, COLOR_RED, 3, 15, False ,chest_part_contour[0], chest_part_contour[1], chest_part_contour[2], chest_part_contour[3])
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    chest_part_img = chest_gui_img[chest_part_y_top:chest_part_y_bottom, chest_part_x_start:chest_part_x_end]

    if debug:
        width = chest_part_img.shape[1]
        height = chest_part_img.shape[0]
        x = int((MONITOR_WIDTH / 2) - (width / 2))
        y = int((MONITOR_HEIGHT / 2) - (height / 2))
        cv2.namedWindow("chest_part_img")
        cv2.moveWindow("chest_part_img", x, y)
        cv2.imshow("chest_part_img", chest_part_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    return (chest_part_img, is_chest_large)

chest_gui_chest_part_img, is_chest_large = crop_chest_gui_chest_part(chest_gui_img, debug=True)

In [6]:
screenshot_image = cv2.imread("images/large_chest.png")

chest_gui_img, corners_of_chest_gui_list = crop_chest_gui(image=screenshot_image, debug=True)
chest_gui_chest_part_img, is_chest_large = crop_chest_gui_chest_part(chest_gui_img, debug=True)

In [8]:
def crop_chest_cells(chest_gui_chest_part, is_chest_large):
    col_count = 9
    row_count = 3
    if is_chest_large:
        row_count = 6

    step_x = round(chest_gui_chest_part.shape[1] / col_count)
    step_y = round(chest_gui_chest_part.shape[0] / row_count)
    
    cells_coords = []

    for y in range(0, row_count):
        row_cells_coords = []
        for x in range(0, col_count):
            start_x = x * step_x
            start_y = y * step_y
            end_x = (x + 1) * step_x
            end_y = (y + 1) * step_y
            row_cells_coords.append((start_x, start_y, end_x, end_y))
        cells_coords.append(row_cells_coords)

    cell_width = cells_coords[0][0][2]
    cell_height = cells_coords[0][0][3]

    cell_top_border_size = round(cell_height / 24)
    cell_start_border_size = round(cell_width / 24)
    cell_bottom_border_size = round(cell_height / 14.4)
    cell_end_border_size = round(cell_width / 14.4)

    cells_img_list = []
    output = chest_gui_chest_part.copy()

    for row_cells_coords in cells_coords:
        for cell_coord in row_cells_coords:
            cell_coord_x_start = cell_coord[0]
            cell_coord_x_end = cell_coord[2]
            cell_coord_y_top = cell_coord[1]
            cell_coord_y_bottom = cell_coord[3]

            cell_inside_coord_x_start = cell_coord_x_start + cell_start_border_size
            cell_inside_coord_x_end = cell_coord_x_end - cell_end_border_size
            cell_inside_coord_y_top = cell_coord_y_top + cell_top_border_size
            cell_inside_coord_y_bottom = cell_coord_y_bottom - cell_bottom_border_size

            cell_contour = np.array([
                [cell_inside_coord_x_start, cell_inside_coord_y_top],
                [cell_inside_coord_x_start, cell_inside_coord_y_bottom],
                [cell_inside_coord_x_end, cell_inside_coord_y_bottom],
                [cell_inside_coord_x_end, cell_inside_coord_y_top]
            ])

            animate_draw_connect_points(output, COLOR_RED, 3, 22, False, cell_contour[0], cell_contour[1], cell_contour[2], cell_contour[3])

            cell_img = chest_gui_chest_part[
                cell_inside_coord_y_top:cell_inside_coord_y_bottom,
                cell_inside_coord_x_start:cell_inside_coord_x_end
            ]

            cells_img_list.append(cell_img)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return cells_img_list

chest_gui_chest_parts_imgs = crop_chest_cells(chest_gui_chest_part_img, is_chest_large)

In [15]:
def get_cell_coords(inventory_img_width, inventory_img_height, corners_of_chest_gui_list,chest_gui_chest_part_width, chest_gui_chest_part_height, is_chest_large):

    if is_chest_large:
        chest_part_y_top = round(inventory_img_height / 13.47692)
    else:
        chest_part_y_top = round(inventory_img_height / 10.15385)
    chest_part_x_start = round(inventory_img_width / 27.84)

    col_count = 9
    row_count = 3
    if is_chest_large:
        row_count = 6

    step_x = round(chest_gui_chest_part_width / col_count)
    step_y = round(chest_gui_chest_part_height / row_count)

    # The center coord of each cell.
    cell_coord_list = []

    # Loop over the grid locations.
    for y in range(0, row_count * 4):
        for x in range(0, col_count):
            # Compute the starting and ending (x, y) coordinates of the current cell.
            x = x * (y + 1)
            start_x = x * step_x
            start_y = y * step_y
            end_x = (x + 1) * step_x
            end_y = (y + 1) * step_y

            chest_top_left_corner_coord = corners_of_chest_gui_list[0]
            cell_center_x = round((start_x + end_x) / 2)
            cell_center_y = round((start_y + end_y) / 2)
            cell_center = (
                cell_center_x + chest_top_left_corner_coord[0] + chest_part_x_start, 
                cell_center_y + chest_top_left_corner_coord[1] + chest_part_y_top)
            cell_coord_list.append(cell_center)
    
    return cell_coord_list

cell_coord_list = get_cell_coords(
    chest_gui_img.shape[1],
    chest_gui_img.shape[0],
    corners_of_chest_gui_list,
    chest_gui_chest_part_img.shape[1],
    chest_gui_chest_part_img.shape[0],
    is_chest_large
)

print("cell_coord_list:", cell_coord_list)

output = screenshot_image.copy()
cv2.imshow("image", output)
cv2.waitKey(1500)
for index, coord in enumerate(cell_coord_list):
    # output = cv2.rectangle(img=output, pt1=(coord[0], coord[1]), pt2=(coord[0] + 1, coord[1] + 1), color=COLOR_RED, thickness=5)
    # print("coordinate " + str(index) + " =", coord)
    animate_draw_point(image=output, color=COLOR_RED, thickness=12, delay_in_millisecond=1, pt=(coord[0], coord[1]))
    
cv2.waitKey(0)
cv2.destroyAllWindows()

cell_coord_list: [(672, 276), (744, 276), (816, 276), (888, 276), (960, 276), (1032, 276), (1104, 276), (1176, 276), (1248, 276), (672, 348), (816, 348), (960, 348), (1104, 348), (1248, 348), (1392, 348), (1536, 348), (1680, 348), (1824, 348), (672, 420), (888, 420), (1104, 420), (1320, 420), (1536, 420), (1752, 420), (1968, 420), (2184, 420), (2400, 420), (672, 492), (960, 492), (1248, 492), (1536, 492), (1824, 492), (2112, 492), (2400, 492), (2688, 492), (2976, 492), (672, 564), (1032, 564), (1392, 564), (1752, 564), (2112, 564), (2472, 564), (2832, 564), (3192, 564), (3552, 564), (672, 636), (1104, 636), (1536, 636), (1968, 636), (2400, 636), (2832, 636), (3264, 636), (3696, 636), (4128, 636), (672, 708), (1176, 708), (1680, 708), (2184, 708), (2688, 708), (3192, 708), (3696, 708), (4200, 708), (4704, 708), (672, 780), (1248, 780), (1824, 780), (2400, 780), (2976, 780), (3552, 780), (4128, 780), (4704, 780), (5280, 780), (672, 852), (1320, 852), (1968, 852), (2616, 852), (3264, 852)

In [19]:
def get_cell_coords(inventory_img_width, inventory_img_height, corners_of_chest_gui_list,chest_gui_chest_part_width, chest_gui_chest_part_height, is_chest_large):

    if is_chest_large:
        chest_part_y_top = round(inventory_img_height / 13.47692)
    else:
        chest_part_y_top = round(inventory_img_height / 10.15385)
    chest_part_x_start = round(inventory_img_width / 27.84)

    col_count = 9
    row_count = 3
    if is_chest_large:
        row_count = 6

    step_x = round(chest_gui_chest_part_width / col_count)
    step_y = round(chest_gui_chest_part_height / row_count)

    # The center coord of each cell.
    cell_coord_list = []

    # Loop over the grid locations.
    for y in range(0, row_count):
        for x in range(0, col_count):
            # Compute the starting and ending (x, y) coordinates of the current cell.
            start_x = x * step_x
            start_y = y * step_y
            end_x = (x + 1) * step_x
            end_y = (y + 1) * step_y

            chest_top_left_corner_coord = corners_of_chest_gui_list[0]
            cell_center_x = round((start_x + end_x) / 2)
            cell_center_y = round((start_y + end_y) / 2)
            cell_center = (
                cell_center_x + chest_top_left_corner_coord[0] + chest_part_x_start, 
                cell_center_y + chest_top_left_corner_coord[1] + chest_part_y_top)
            cell_coord_list.append(cell_center)
    
    return cell_coord_list

cell_coord_list = get_cell_coords(
    chest_gui_img.shape[1],
    chest_gui_img.shape[0],
    corners_of_chest_gui_list,
    chest_gui_chest_part_img.shape[1],
    chest_gui_chest_part_img.shape[0],
    is_chest_large
)

# print("cell_coord_list:", cell_coord_list)

output = screenshot_image.copy()
cv2.imshow("image", output)
cv2.waitKey(1500)
for index, coord in enumerate(cell_coord_list):
    # output = cv2.rectangle(img=output, pt1=(coord[0], coord[1]), pt2=(coord[0] + 1, coord[1] + 1), color=COLOR_RED, thickness=5)
    print("cell " + str(index) + " coordinate =", coord)
    animate_draw_point(image=output, color=COLOR_RED, thickness=12, delay_in_millisecond=1, pt=(coord[0], coord[1]))
    
cv2.waitKey(0)
cv2.destroyAllWindows()

cell 0 coordinate = (672, 276)
cell 1 coordinate = (744, 276)
cell 2 coordinate = (816, 276)
cell 3 coordinate = (888, 276)
cell 4 coordinate = (960, 276)
cell 5 coordinate = (1032, 276)
cell 6 coordinate = (1104, 276)
cell 7 coordinate = (1176, 276)
cell 8 coordinate = (1248, 276)
cell 9 coordinate = (672, 348)
cell 10 coordinate = (744, 348)
cell 11 coordinate = (816, 348)
cell 12 coordinate = (888, 348)
cell 13 coordinate = (960, 348)
cell 14 coordinate = (1032, 348)
cell 15 coordinate = (1104, 348)
cell 16 coordinate = (1176, 348)
cell 17 coordinate = (1248, 348)
cell 18 coordinate = (672, 420)
cell 19 coordinate = (744, 420)
cell 20 coordinate = (816, 420)
cell 21 coordinate = (888, 420)
cell 22 coordinate = (960, 420)
cell 23 coordinate = (1032, 420)
cell 24 coordinate = (1104, 420)
cell 25 coordinate = (1176, 420)
cell 26 coordinate = (1248, 420)
